In [1]:
pip install selenium

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import requests
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rtc-directory')
time.sleep(3)
driver.maximize_window()
redbus_dic= driver.find_element(By.CSS_SELECTOR,'a[href="/online-booking/astc"]')
redbus_dic .click()

ASTC_route_names=[]
ASTC_route_links=[] 
def scrape_data(): 
    route_names= driver.find_elements (By.CLASS_NAME, "route") 
    for route_name in route_names: 
        ASTC_route_names.append(route_name.text)
    route_links= driver.find_elements (By.CLASS_NAME, "route")
    hrefs =[route_link.get_attribute('href') for route_link in route_links] 
    for href in hrefs:
        ASTC_route_links.append(href)
page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")

print(len(page_tabs))



for i in range(1, len(page_tabs)+1):
    if i > 0:
        x_path = f'/html/body/div[1]/div/div[4]/div[12]/div[{i}]' 
        element=driver.find_element(By.XPATH, x_path)
        driver.execute_script("arguments[0].click();", element)
        scrape_data()
route={
    'route name':ASTC_route_names,
    'route link':ASTC_route_links
}   
df=pd.DataFrame(route)     
print(df)




5
                        route name  \
0               Tezpur to Guwahati   
1               Guwahati to Tezpur   
2       Nagaon (Assam) to Guwahati   
3       Guwahati to Nagaon (Assam)   
4             Goalpara to Guwahati   
5        Jorhat to North Lakhimpur   
6               Dhubri to Guwahati   
7               Guwahati to Dhubri   
8      North Lakhimpur to Sibsagar   
9        North Lakhimpur to Jorhat   
10          Dhekiajuli to Guwahati   
11             Jorhat to Dibrugarh   
12               Jorhat to Dhemaji   
13     Sibsagar to North Lakhimpur   
14               Dhemaji to Jorhat   
15             Tezpur to Dibrugarh   
16             Haflong to Guwahati   
17    North Lakhimpur to Dibrugarh   
18              Jorhat to Tinsukia   
19             Dibrugarh to Tezpur   
20   Guwahati to Biswanath Charali   
21             Guwahati to Haflong   
22    Dibrugarh to North Lakhimpur   
23       Nagaon (Assam) to Haflong   
24                 Tezpur to Moran   
25        

In [4]:
bus_names = []
bus_types = []
bus_rating = []
dep_time = []
avl_time = []
duration = []
available_seat = []
price = []
route_names = []
route_links = []
def scroll_down():
   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
   time.sleep(1)


for route_name, route_link in zip(ASTC_route_names,ASTC_route_links):
    driver.get(route_link)
    time.sleep(1)
    for details in range(3):
        try:
            view_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'button')))
            view_more_button.click()
        except Exception as e:
            print(f"View more button not found for {route_link}: {e}")
            continue
   
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
           scroll_down()
           new_height = driver.execute_script("return document.body.scrollHeight")
           if new_height == last_height:
              break
           last_height = new_height


        
        bus_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_type_elements = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
        rating_elements = driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')
        depature_elements = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
        arrival_elements = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
        duration_elements = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
        seat_elements = driver.find_elements(By.XPATH, '//div[@class="column-eight w-15 fl"]')
        rate_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')

        for bus in bus_name_elements:
            bus_names.append(bus.text)
            route_names.append(route_name)
            route_links.append(route_link)
        for bus_type in bus_type_elements:
            bus_types.append(bus_type.text)
        for rating in rating_elements:
            bus_rating.append(rating.text)
        for dep in depature_elements:
            dep_time.append(dep.text)
        for arr in arrival_elements:
            avl_time.append(arr.text)
        for dur in duration_elements:
            duration.append(dur.text)
        for seat in seat_elements:
            available_seat.append(seat.text)
        for fare in rate_elements:
            price.append(fare.text)



max_length = max(len(bus_names), len(bus_types), len(bus_rating), len(dep_time), len(avl_time), len(duration), len(available_seat), len(price))

bus_names.extend([None] * (max_length - len(bus_names)))
bus_types.extend([None] * (max_length - len(bus_types)))
bus_rating.extend([None] * (max_length - len(bus_rating)))
dep_time.extend([None] * (max_length - len(dep_time)))
avl_time.extend([None] * (max_length - len(avl_time)))
duration.extend([None] * (max_length - len(duration)))
available_seat.extend([None] * (max_length - len(available_seat)))
price.extend([None] * (max_length - len(price)))


data = {
    "Route Name": route_names,
    "Route Link": route_links,
    'Bus Name': bus_names,
    'Bus Type': bus_types,
    'Bus Rating': bus_rating,
    'Departure Time': dep_time,
    'Arrival Time': avl_time,
    'Duration': duration,
    'Available Seats': available_seat,
    'Price': price
}

df = pd.DataFrame(data)
df.to_csv('assam_bus_details.csv', index=False)
print(df.head())


View more button not found for https://www.redbus.in/bus-tickets/tezpur-to-guwahati: Message: element click intercepted: Element <div class="button">...</div> is not clickable at point (1442, 15). Other element would receive the click: <div class="modify-sec-onward clearfix modify-sec d-color f-bold">...</div>
  (Session info: chrome=126.0.6478.182)
Stacktrace:
	GetHandleVerifier [0x00007FF749D0EEB2+31554]
	(No symbol) [0x00007FF749C87EE9]
	(No symbol) [0x00007FF749B4872A]
	(No symbol) [0x00007FF749BA012E]
	(No symbol) [0x00007FF749B9DAF2]
	(No symbol) [0x00007FF749B9AF8B]
	(No symbol) [0x00007FF749B9A156]
	(No symbol) [0x00007FF749B8C151]
	(No symbol) [0x00007FF749BBD02A]
	(No symbol) [0x00007FF749B8BA76]
	(No symbol) [0x00007FF749BBD240]
	(No symbol) [0x00007FF749BDC977]
	(No symbol) [0x00007FF749BBCDD3]
	(No symbol) [0x00007FF749B8A33B]
	(No symbol) [0x00007FF749B8AED1]
	GetHandleVerifier [0x00007FF74A018B2D+3217341]
	GetHandleVerifier [0x00007FF74A065AF3+3532675]
	GetHandleVerifier

: 

In [15]:

bus_names=[]
bus_types=[]
bus_rating=[]
dep_time=[]
avl_time=[]
duration=[]
available_seat=[]
price=[]
for route in APSRTC_route_links:
    driver.get(route)
    time.sleep(2)
    
    view_more_button = WebDriverWait(driver, 10).until( EC.element_to_be_clickable((By.CLASS_NAME, 'button')))        
    for button in view_more_button:
        button.click()
        time.sleep(2)
    
    bus_name=driver.find_elements(By.CSS_SELECTOR,'div[class="travels lh-24 f-bold d-color"]')
    bus_type=driver.find_elements(By.XPATH,'//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
    rating=driver.find_elements(By.XPATH,'//div[@class="rating-sec lh-24"]')
    depature=driver.find_elements(By.XPATH,'//div[@class="dp-time f-19 d-color f-bold"]')
    arival=driver.find_elements(By.XPATH,'//div[@class="bp-time f-19 d-color disp-Inline"]')
    time_take=driver.find_elements(By.XPATH,'//div[@class="dur l-color lh-24"]')
    seat=driver.find_elements(By.XPATH,'//div[@class="seat-left m-top-16 "]')
    rate=driver.find_elements(By.XPATH,'//span[@class="f-19 f-bold"]')
    for bus in bus_name:
        bus_names.append(bus.text)
    for type in bus_type:
        bus_types.append(type.text)
    for ratinges in rating:
        bus_rating.append(ratinges.text)
    for start in depature:
        dep_time.append(start.text)
    for end in arival:
        avl_time.append(end.text)
    for time_takes in time_take:
        duration.append(time_takes.text)
    for avail_seat in seat:
        available_seat.append(avail_seat.text)
    for fare in rate:
        price.append(fare.text) 

print(bus_names)
print(bus_types) 
print(bus_rating)
print(dep_time)
print(avl_time)
print(duration)
print(available_seat,price)

['APSRTC - 35271', 'APSRTC - 3585', 'APSRTC - 35133', 'APSRTC - 35135', '', '', '', '', '', '', 'Svkdt travels', 'IntrCity SmartBus', '', '', '', 'APSRTC - 24279', 'APSRTC - 2891', 'APSRTC - 2534', 'APSRTC - 2815', '', '', '', 'APSRTC - 27092', 'APSRTC - 23532', 'APSRTC - 2201', '', '', '', '', '', '', '', 'APSRTC - 9007', 'APSRTC - 9049', 'APSRTC - 9888', 'APSRTC - 9003', '', '', '', '', '', '', 'APSRTC - 50144', 'APSRTC - 6018', 'APSRTC - 6010', '', '', '', '', '', '', '', 'APSRTC - 52935', 'APSRTC - 52267', 'APSRTC - 6745', 'APSRTC - 52937', '', '', '', '', '', '', 'APSRTC - 5518', 'APSRTC - 9003', 'APSRTC - 9041', '', '', '', '', '', '', '', 'APSRTC - 9478', 'APSRTC - 3383', 'APSRTC - 3365', '', '', '', '', '', '', '', 'APSRTC - 5380', 'APSRTC - 5866', 'APSRTC - 5913', 'APSRTC - 45901', '', '', '', '', '', '', 'APSRTC - 2638', 'APSRTC - 3879', 'APSRTC - 5467', '', '', '', '', '', '', '', 'APSRTC - 4430', 'APSRTC - 36005', 'APSRTC - 4406', 'APSRTC - 4418', '', '', '', '', '', 'APSRT

In [16]:
print(bus_names)
len(bus_names)

['APSRTC - 35271', 'APSRTC - 3585', 'APSRTC - 35133', 'APSRTC - 35135', '', '', '', '', '', '', 'Svkdt travels', 'IntrCity SmartBus', '', '', '', 'APSRTC - 24279', 'APSRTC - 2891', 'APSRTC - 2534', 'APSRTC - 2815', '', '', '', 'APSRTC - 27092', 'APSRTC - 23532', 'APSRTC - 2201', '', '', '', '', '', '', '', 'APSRTC - 9007', 'APSRTC - 9049', 'APSRTC - 9888', 'APSRTC - 9003', '', '', '', '', '', '', 'APSRTC - 50144', 'APSRTC - 6018', 'APSRTC - 6010', '', '', '', '', '', '', '', 'APSRTC - 52935', 'APSRTC - 52267', 'APSRTC - 6745', 'APSRTC - 52937', '', '', '', '', '', '', 'APSRTC - 5518', 'APSRTC - 9003', 'APSRTC - 9041', '', '', '', '', '', '', '', 'APSRTC - 9478', 'APSRTC - 3383', 'APSRTC - 3365', '', '', '', '', '', '', '', 'APSRTC - 5380', 'APSRTC - 5866', 'APSRTC - 5913', 'APSRTC - 45901', '', '', '', '', '', '', 'APSRTC - 2638', 'APSRTC - 3879', 'APSRTC - 5467', '', '', '', '', '', '', '', 'APSRTC - 4430', 'APSRTC - 36005', 'APSRTC - 4406', 'APSRTC - 4418', '', '', '', '', '', 'APSRT

468

In [17]:
print(bus_names)
print(bus_types) 
print(bus_rating)
print(dep_time)
print(avl_time)
print(duration)
print(available_seat,price)


['APSRTC - 35271', 'APSRTC - 3585', 'APSRTC - 35133', 'APSRTC - 35135', '', '', '', '', '', '', 'Svkdt travels', 'IntrCity SmartBus', '', '', '', 'APSRTC - 24279', 'APSRTC - 2891', 'APSRTC - 2534', 'APSRTC - 2815', '', '', '', 'APSRTC - 27092', 'APSRTC - 23532', 'APSRTC - 2201', '', '', '', '', '', '', '', 'APSRTC - 9007', 'APSRTC - 9049', 'APSRTC - 9888', 'APSRTC - 9003', '', '', '', '', '', '', 'APSRTC - 50144', 'APSRTC - 6018', 'APSRTC - 6010', '', '', '', '', '', '', '', 'APSRTC - 52935', 'APSRTC - 52267', 'APSRTC - 6745', 'APSRTC - 52937', '', '', '', '', '', '', 'APSRTC - 5518', 'APSRTC - 9003', 'APSRTC - 9041', '', '', '', '', '', '', '', 'APSRTC - 9478', 'APSRTC - 3383', 'APSRTC - 3365', '', '', '', '', '', '', '', 'APSRTC - 5380', 'APSRTC - 5866', 'APSRTC - 5913', 'APSRTC - 45901', '', '', '', '', '', '', 'APSRTC - 2638', 'APSRTC - 3879', 'APSRTC - 5467', '', '', '', '', '', '', '', 'APSRTC - 4430', 'APSRTC - 36005', 'APSRTC - 4406', 'APSRTC - 4418', '', '', '', '', '', 'APSRT

In [28]:
data={
    'bus name': [bus_names],
    'bus type': [bus_types],
    'bus rating': [bus_rating],
    'dep time' : [dep_time],
    'avl time': [avl_time],
    'duration': [duration],
    'avail seat': [avail_seat],
    'price':[price]
    }

df = pd.DataFrame(data)
df.to_csv('APSRTC_all_bus_details.csv', index=False)
print(df.head())

  bus name bus type bus rating dep time avl time duration  \
0       []       []         []       []       []       []   

                                          avail seat price  
0  <selenium.webdriver.remote.webelement.WebEleme...    []  
